In [1]:
# pip install tslearn

import tslearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
 
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# 시군구 위치

In [2]:
df_sgg = pd.read_csv("location_final.csv", encoding = 'ansi')

In [3]:
df_sgg.drop(['Unnamed: 0'], axis = 1)

,loc_name,lon,lat,emd,sgg,field,barren,agri_land,forest,water,wet_land,used_area,grass,total
0,7공구생활폐기물집하시설,126.662504,37.383067,인천 연수구 송도동,연수구,인천,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,가정공원,126.674096,37.520941,인천 서구 가정동,서구,인천,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,가좌완충녹지,126.684497,37.484762,인천 서구 가좌동,서구,인천,0.028684,0.001027,0.0,0.0,0.0,0.684913,0.285376,1.0
3,갑곶2리마을회관,126.510187,37.737728,인천 강화군 강화읍,강화군,강화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,강화군보건소,126.485095,37.736896,인천 강화군 강화읍,강화군,강화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,화도보건지소,126.420558,37.631868,인천 강화군 화도면,강화군,강화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,화문석문화관,126.454404,37.795779,인천 강화군 송해면,강화군,강화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,황청보건진료소,126.361481,37.721230,인천 강화군 내가면,강화군,강화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,효성119안전센터,126.706543,37.527374,인천 계양구 효성동,계양구,인천,0.000000,0.000000,0.0,0.0,0.0,0.994116,0.005884,1.0


# 날씨

In [5]:
df_weather = pd.read_csv("인천 일별 기상 데이터.csv", encoding = 'ansi', engine = 'python')
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   지점          4200 non-null   int64  
 1   지점명         4200 non-null   object 
 2   일시          4200 non-null   object 
 3   평균기온(°C)    4200 non-null   float64
 4   최저기온(°C)    4200 non-null   float64
 5   최고기온(°C)    4200 non-null   float64
 6   일강수량(mm)    1440 non-null   float64
 7   평균 풍속(m/s)  4200 non-null   float64
 8   평균 상대습도(%)  4200 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 295.4+ KB


In [6]:
df_weather.head()

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%)
0,102,백령도,4/5/2016,9.7,7.2,13.7,NaN,6.0,56.3
1,102,백령도,4/6/2016,10.2,6.3,14.6,NaN,4.7,59.0
2,102,백령도,4/7/2016,9.7,7.5,12.9,NaN,3.7,77.9
3,102,백령도,4/8/2016,9.8,4.5,15.7,NaN,3.8,76.0
4,102,백령도,4/9/2016,7.9,5.3,11.1,NaN,3.8,95.9


In [7]:
# column 이름 변경  # 한번만 sheft enter , 두번하면 에러생김 
df_weather = df_weather.drop(['지점'], axis = 1)
df_weather.set_axis(['field', 'date', 'avg_temp', 'min_temp', 'max_temp', 'rainfall', 'avg_wind', 'avg_humi'], 
                    axis = 'columns', inplace = True)
df_weather = df_weather.sort_values('date')
df_weather.loc[df_weather['rainfall'].isna()]=0
df_weather['date'] = pd.to_datetime(df_weather['date'])

df_weather.head()

,field,date,avg_temp,min_temp,max_temp,rainfall,avg_wind,avg_humi
4010,0,1970-01-01,0.0,0.0,0.0,0.0,0.0,0.0
3915,0,1970-01-01,0.0,0.0,0.0,0.0,0.0,0.0
4105,0,1970-01-01,0.0,0.0,0.0,0.0,0.0,0.0
2881,강화,2017-01-01,2.0,-4.3,7.3,0.0,0.8,83.0
1576,0,1970-01-01,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_weather_ic = df_weather.loc[df_weather['field'] == '인천']
df_weather_kh = df_weather.loc[df_weather['field'] == '강화']
df_weather_br = df_weather.loc[df_weather['field'] == '백령도']

### 인천, 강화, 백령도로 분리

In [10]:
df_weather_ic = df_weather_ic.copy()

df_weather_ic['sma14_temp'] = (df_weather_ic.avg_temp.rolling(15).sum() - df_weather_ic.avg_temp)/14
df_weather_ic['sma14_wind'] = (df_weather_ic.avg_wind.rolling(15).sum() - df_weather_ic.avg_wind)/14
df_weather_ic['sma14_humi'] = (df_weather_ic.avg_humi.rolling(15).sum() - df_weather_ic.avg_humi)/14
df_weather_ic['cum28_rain'] = df_weather_ic.rainfall.rolling(29).sum() - df_weather_ic.rainfall.rolling(14).sum()

In [12]:
df_weather_kh = df_weather_kh.copy()

df_weather_kh['sma14_temp'] = (df_weather_kh.avg_temp.rolling(15).sum() - df_weather_kh.avg_temp)/14
df_weather_kh['sma14_wind'] = (df_weather_kh.avg_wind.rolling(15).sum() - df_weather_kh.avg_wind)/14
df_weather_kh['sma14_humi'] = (df_weather_kh.avg_humi.rolling(15).sum() - df_weather_kh.avg_humi)/14
df_weather_kh['cum28_rain'] = df_weather_kh.rainfall.rolling(29).sum() - df_weather_kh.rainfall.rolling(14).sum()

In [13]:
df_weather_br = df_weather_br.copy()


df_weather_br['sma14_temp'] = (df_weather_br.avg_temp.rolling(15).sum() - df_weather_br.avg_temp)/14
df_weather_br['sma14_wind'] = (df_weather_br.avg_wind.rolling(15).sum() - df_weather_br.avg_wind)/14
df_weather_br['sma14_humi'] = (df_weather_br.avg_humi.rolling(15).sum() - df_weather_br.avg_humi)/14
df_weather_br['cum28_rain'] = df_weather_br.rainfall.rolling(29).sum() - df_weather_br.rainfall.rolling(14).sum()

# 모기 데이터

In [14]:
df_mosquito = pd.read_csv('인천모기데이터_소분류.csv', encoding = 'ansi' , engine = 'python')
df_mosquito

,Unnamed: 0,type,id,geometry/type,geometry/coordinates/0,geometry/coordinates/1,properties/OBJECTID,properties/MOS_DAY,properties/LOC_NM,properties/SUM_MOS_CNT,year,대분류,소분류
0,0,Feature,34695860,Point,126.635042,37.468414,34695860,2016-04-18 00:00:00,보건환경연구원,0,2016-04-18,연구원,학교
1,1,Feature,34695861,Point,126.664151,37.463157,34695861,2016-04-19 00:00:00,수봉공원,0,2016-04-19,공원,공원
2,2,Feature,34695862,Point,126.671372,37.419236,34695862,2016-04-19 00:00:00,연수구보건소,0,2016-04-19,보건소,보건소
3,3,Feature,34695863,Point,126.724178,37.498668,34695863,2016-04-19 00:00:00,부평구보건소,2,2016-04-19,보건소,보건소
4,4,Feature,34695864,Point,126.625803,37.483148,34695864,2016-04-19 00:00:00,동구보건소,0,2016-04-19,보건소,보건소
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46832,46832,Feature,34742178,Point,126.673960,37.544535,34742178,2019-10-28 00:00:00,서구보건소,6,2019-10-28,보건소,보건소
46833,46833,Feature,34742179,Point,126.491310,37.640566,34742179,2019-10-28 00:00:00,길상면보건지소,6,2019-10-28,보건소,보건소
46834,46834,Feature,34742180,Point,126.237790,37.771217,34742180,2019-10-28 00:00:00,난정보건진료소,5,2019-10-28,보건소,보건소
46835,46835,Feature,34742181,Point,126.411942,37.774615,34742181,2019-10-28 00:00:00,하점면보건지소,5,2019-10-28,보건소,보건소


In [15]:
# column 이름 변경
df_mosquito = df_mosquito.drop(['Unnamed: 0', 'type', 'id', 'geometry/type', 'properties/OBJECTID', 'properties/MOS_DAY', '대분류', '소분류'], axis = 1)
df_mosquito.set_axis(['lon', 'lat', 'loc_name', 'mosquito', 'date'], axis = 'columns', inplace = True)

df_mosquito['date'] = pd.to_datetime(df_mosquito['date']) 

In [16]:
df_new = pd.merge(df_mosquito,df_sgg, how = 'inner', on = 'loc_name')
df_new = df_new.drop(['Unnamed: 0', 'lon_y', 'lat_y'], axis = 1)

In [17]:
df_new = pd.merge(df_mosquito,df_sgg, how = 'right', on = 'loc_name')
df_new = df_new.drop(['Unnamed: 0', 'lon_y', 'lat_y'], axis = 1)

In [18]:
df_new.set_axis(['lon', 'lat', 'loc_name', 'mosquito', 'date', 'emd', 'sgg', 'field', 'barren', 'agri_land', 'forest', 'water', 'wet_land', 'used_area', 'grass', 'total'], axis = 'columns', inplace = True)

In [19]:
df_new_ic = df_new.loc[df_new['field'] == '인천']
df_new_kh = df_new.loc[df_new['field'] == '강화']
df_new_br = df_new.loc[df_new['field'] == '백령도']

In [20]:
df_new_ic = pd.merge(df_new_ic, df_weather_ic, how = "inner", on = 'date')
df_new_kh = pd.merge(df_new_kh, df_weather_kh, how = "inner", on = 'date')
df_new_br = pd.merge(df_new_br, df_weather_br, how = "inner", on = 'date')

In [21]:
df_final = pd.concat([df_new_ic, df_new_kh, df_new_br])
df_final.drop(['field_x', 'field_y'], axis = 1, inplace = True)

In [22]:
df_final

,lon,lat,loc_name,mosquito,date,emd,sgg,barren,agri_land,forest,...,avg_temp,min_temp,max_temp,rainfall,avg_wind,avg_humi,sma14_temp,sma14_wind,sma14_humi,cum28_rain
0,126.662504,37.383067,7공구생활폐기물집하시설,22,2019-09-07,인천 연수구 송도동,연수구,NaN,NaN,NaN,...,25.2,22.5,29.5,1.4,8.6,80.3,22.492857,2.564286,78.871429,48.7
1,126.674096,37.520941,가정공원,44,2019-09-07,인천 서구 가정동,서구,NaN,NaN,NaN,...,25.2,22.5,29.5,1.4,8.6,80.3,22.492857,2.564286,78.871429,48.7
2,126.684497,37.484762,가좌완충녹지,7,2019-09-07,인천 서구 가좌동,서구,0.028684,0.001027,0.000000,...,25.2,22.5,29.5,1.4,8.6,80.3,22.492857,2.564286,78.871429,48.7
3,126.423605,37.429422,거잠포마을회관,534,2019-09-07,인천 중구 덕교동,중구,0.005354,0.200060,0.514061,...,25.2,22.5,29.5,1.4,8.6,80.3,22.492857,2.564286,78.871429,48.7
4,126.689412,37.616873,검단2동주민센터,3,2019-09-07,인천 서구 불로동,서구,0.011048,0.205693,0.328424,...,25.2,22.5,29.5,1.4,8.6,80.3,22.492857,2.564286,78.871429,48.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,126.461471,37.253087,영흥보건지소,3,2017-10-18,인천 옹진군 영흥면,옹진군,0.010337,0.533264,0.289378,...,14.9,13.3,16.3,0.0,4.6,76.0,7.971429,4.757143,72.421429,11.8
418,126.461471,37.253087,영흥보건지소,6,2017-10-23,인천 옹진군 영흥면,옹진군,0.010337,0.533264,0.289378,...,13.4,11.6,17.3,0.0,3.5,66.6,14.821429,4.321429,83.128571,11.7
419,126.461471,37.253087,영흥보건지소,16,2019-05-26,인천 옹진군 영흥면,옹진군,0.010337,0.533264,0.289378,...,17.4,14.9,20.2,0.8,7.3,87.0,14.264286,5.021429,90.171429,29.7
420,126.461471,37.253087,영흥보건지소,14,2019-05-27,인천 옹진군 영흥면,옹진군,0.010337,0.533264,0.289378,...,14.2,12.1,17.0,10.5,6.6,90.4,14.700000,5.107143,89.742857,52.7


In [23]:
df_final.to_csv("df_final.csv", encoding = 'ansi')